# Credit Card Clustering

Bu projede, müşterilerin kredi kartı kullanım alışkanlıklarını analiz ederek onları farklı segmentlere ayıracağız (Kümeleme).
Yapılan İyileştirmeler:
1. **Veri Ön İşleme**: Eksik verilerin temizlenmesi ve `StandardScaler` ile ölçeklendirme.
2. **Elbow Yöntemi**: Optimal küme sayısının belirlenmesi.
3. **K-Means Kümeleme**: Müşterilerin gruplandırılması.
4. **Dendrogram**: Hiyerarşik yapının görselleştirilmesi.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import scipy.cluster.hierarchy as shc
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

# Veriyi Yükle
data = pd.read_csv("CC GENERAL.csv")
print("Veri Boyutu:", data.shape)
print(data.head())

## Veri Temizleme ve Özellik Seçimi
Müşteri ID'sini çıkarıyoruz ve eksik verileri temizliyoruz.

In [ ]:
# CUST_ID gereksiz olduğu için çıkarıyoruz
if 'CUST_ID' in data.columns:
    data = data.drop('CUST_ID', axis=1)

# Eksik verileri kontrol et
print("Eksik Değerler:\n", data.isnull().sum())

# Eksik verileri temizle (veya doldur)
data = data.dropna()
print("Temizlenmiş Veri Boyutu:", data.shape)

## Ölçeklendirme
Kümeleme algoritmaları mesafeye dayalı çalıştığı için verileri aynı ölçeğe getiriyoruz.

In [ ]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

## Elbow Yöntemi (Optimal Küme Sayısı)
Küme sayısına karar vermek için Inertia grafiğini çiziyoruz.

In [ ]:
wcss = []
k_range = range(1, 15)
for k in k_range:
    kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42)
    kmeans.fit(data_scaled)
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(k_range, wcss, marker='o', linestyle='--')
plt.title('Elbow Method')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS (Inertia)')
plt.show()

## K-Means Kümeleme
Elbow grafiğinde kırılma noktasına göre (genellikle 4-6 arası) 5 küme seçiyoruz.

In [ ]:
kmeans = KMeans(n_clusters=5, init='k-means++', random_state=42)
clusters = kmeans.fit_predict(data_scaled)
data['CREDIT_CARD_SEGMENTS'] = clusters

# Kümeleri isimlendir
data['Cluster Name'] = data['CREDIT_CARD_SEGMENTS'].apply(lambda x: f"Cluster {x+1}")
print(data['Cluster Name'].value_counts())

## 3D Görselleştirme
Balance, Purchases ve Credit Limit özelliklerine göre kümeleri görselleştiriyoruz.

In [ ]:
PLOT = go.Figure()
for i in list(data["CREDIT_CARD_SEGMENTS"].unique()):
    PLOT.add_trace(go.Scatter3d(
        x = data[data["CREDIT_CARD_SEGMENTS"]== i]['BALANCE'],
        y = data[data["CREDIT_CARD_SEGMENTS"] == i]['PURCHASES'],
        z = data[data["CREDIT_CARD_SEGMENTS"] == i]['CREDIT_LIMIT'],                        
        mode = 'markers',
        marker_size = 5,
        marker_line_width = 1,
        name = f"Cluster {i+1}"
    ))

PLOT.update_traces(hovertemplate='BALANCE: %{x} <br>PURCHASES: %{y} <br>CREDIT_LIMIT: %{z}')
PLOT.update_layout(
    width = 900, 
    height = 800, 
    autosize = True, 
    showlegend = True,
    scene = dict(
        xaxis=dict(title = 'BALANCE'),
        yaxis=dict(title = 'PURCHASES'),
        zaxis=dict(title = 'CREDIT_LIMIT')
    ),
    title = "Credit Card Customer Segments 3D"
)
PLOT.show()

## Dendrogram (Hiyerarşik Kümeleme)
Veri seti büyük olduğu için örneklem (sample) üzerinden hiyerarşik ilişkiyi gösteriyoruz.

In [ ]:
# Veri seti çok büyükse dendrogram yavaş olabilir, bu yüzden örneklem alıyoruz
sample_data = data_scaled[:200]  # İlk 200 veri noktası

plt.figure(figsize=(15, 8))
plt.title("Credit Card Customers Dendrogram (Sample)")
dend = shc.dendrogram(shc.linkage(sample_data, method='ward'))
plt.xlabel('Customers')
plt.ylabel('Euclidean Distances')
plt.show()

## Sonuç ve Açıklama

Bu çalışmada Credit Card veri seti üzerinde kümeleme analizi yapılmıştır. Yapılan geliştirmeler:

1.  **Doğru Ölçeklendirme**: Tüm sayısal veriler `StandardScaler` ile ölçeklendirilmiştir. Önceki sürümde sadece 3 sütun kullanılıyor ve ölçeklendirme hatalıydı.
2.  **Kapsamlı Analiz**: Sadece 3 sütun yerine tüm veri seti (CUST_ID hariç) kümeleme algoritmasına dahil edilmiştir. Bu sayede `CASH_ADVANCE`, `PAYMENTS` gibi önemli özellikler de değerlendirilmiştir.
3.  **Elbow Yöntemi**: Optimal küme sayısını belirlemek için bilimsel bir yöntem eklenmiştir.
4.  **Dendrogram**: Verilerin hiyerarşik yapısını anlamak için dendrogram grafiği eklenmiştir.
5.  **Görselleştirme**: 3D kümeleme grafiği daha açıklayıcı hale getirilmiştir.